<font size = 6><h1 align = center>Making Galaxy Populations </h1 ></font>      

<font size = 4><h2 align = center> Derek Sikorski </font><h2>

---
---
---


<font size = 5><h1 align = center>File Summary</h1 ></font>      

**Purpose:** 
This file is meant to construct galaxy population objects based off the the MCed data for Hyperion.

**Outputs:**
This file will output the final GalPop objects after some cuts have been made. Broadly speaking, the final GalPop objects should have:
1. Galaxy locations (Ra, Dec, z)
2. Magnitudes
3. Physical properties from LePhare

Note that halfway through the file I take an output and run LePhare on it, so the file cannot be run as a continuous Python script

---
---
---


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from tqdm.notebook import tqdm
from GalPop import GalPop

<font size = 6><h1 align = center>Loading Data </h1 ></font>      

In [3]:
# COSMOS CATALOG
cosmos_file = fits.open(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/COSMOS2020_CLASSIC_R1_v2.0.fits")
c20p = cosmos_file[1].data

# MCed Redshifts
MCzs = np.load(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/C20_tot100.npy")
MCchoice = np.load(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/C20_totChoice.npy")

print("C20 Catalog shape: ", c20p.shape)
print("MC Catalog shape: ", MCzs.shape)
print("MC Choice Shape: ", MCchoice.shape)

C20 Catalog shape:  (1720700,)
MC Catalog shape:  (1720700, 102)
MC Choice Shape:  (43699, 102)


---
---
---


<font size = 6><h1 align = center>Pack Initial GalPops </h1 ></font>      

The first step is to pack the GalPops with all of the COSMOS things. The masses for these objects will not be correct but will eventually be replaced in LePhare at some point in the code. So this initial step is simply packing GalPops according to some cuts

In [3]:
## Define Constraints ##
ra_range = (149.6, 150.52)  
dec_range = (1.74, 2.73)
z_range = (2., 3.)
IRAC_cut = 25.4
lp_types = [0,2]

# --------------------------------

gps = []        # Where to store the galaxy populations


# Loop through each MC noting col 1 is COSMOS_ID and col 2 is the non-MCed data
for _mc in tqdm(range(1, MCzs.shape[1])):

    # Constrain the galaxy sample
    good_gals = np.where( 
                    (c20p["ALPHA_J2000"] >= ra_range[0]) & (c20p["ALPHA_J2000"] <= ra_range[1])       # RA Check
                    & (c20p["DELTA_J2000"] >= dec_range[0]) & (c20p["DELTA_J2000"] <= dec_range[1])      # Dec Check
                    & (MCzs[:,_mc] >= z_range[0]) & (MCzs[:,_mc] <= z_range[1])           # Redshift Check
                    & ((c20p["IRAC_CH1_MAG"] <= IRAC_cut) | (c20p["IRAC_CH2_MAG"] <= IRAC_cut))  # Check either IRAC1 or IRAC2
                    & (np.isin(c20p['lp_type'], lp_types))      # Check type
                         )     
    
    gs = c20p[good_gals]        # Galaxy data from cosmos
   # Create a mask for matching galaxy IDs in MCchoice
    match_mask = np.isin(gs["ID"], MCchoice[:, 0])

    # Get indices where IDs match
    matching_indices = np.searchsorted(MCchoice[:, 0], gs["ID"], sorter=np.argsort(MCchoice[:, 0]))
    valid_matches = MCchoice[:, 0][matching_indices] == gs["ID"]

    # Use matching_indices to get corresponding z values and set others to 0
    z_type = np.zeros(len(gs), dtype=MCchoice[:, _mc].dtype)  # Preallocate
    z_type[match_mask & valid_matches] = MCchoice[:, _mc][matching_indices[match_mask & valid_matches]]
    
    gpop = GalPop(IDs = gs["ID"],
                  coords = np.c_[gs['ALPHA_J2000'], gs["DELTA_J2000"], MCzs[:,_mc][good_gals]],
                  ms = gs['lp_mass_med'],
                  obs_type= z_type,
                  mags = {'IRAC':np.min(np.c_[gs["IRAC_CH1_MAG"], gs["IRAC_CH2_MAG"]], axis=1)}
                  )
    
    gps.append(gpop)

  0%|          | 0/101 [00:00<?, ?it/s]

In [4]:
gp_folder = 'OldGPs/'

for _id, gp in enumerate(gps):
    gp.saveFile(gp_folder + f'MC_{_id}')

Great, now we're ready to fit the masses...

---
---
---


<font size = 6><h1 align = center>Assigning Peaks </h1 ></font>      

At this point, the masses should have been fit with LePhare and packed into a new folder. Thus, we just need to assign each galaxy to a peak number.

In [2]:
## Load New GalPops ##
new_gps = [f"./LP_GalPops/MC_{idx}.npy" for idx in range(0,101)]

## Load Overdensity Info ##

# Interpolated sigma cube
sig_cube = fits.open(r"C:\Users\sikor\OneDrive\Desktop\benpy\FinalTry\Hyperion_interpolated_cube_sigma.fits")

# Path for folders with .dat sigma files
sigPath = r"C:\Users\sikor\OneDrive\Desktop\BigData\Hyperion\HyperionPeaks"

# Define sigma information
sigs = [2.0, 2.25, 2.3, 2.35, 2.4, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
sig_path = sigPath

sig_sums = [sigPath + f"\peaks_sig{s}.dat" for s in sigs]
sig_folders = [sigPath+ f"\sig{s}_peaks" for s in sigs]

# Assign peaks to objects
for _id, ngp in tqdm(enumerate(new_gps), total=len(new_gps)):

    gp = GalPop.loadFile(ngp)
    gp.assignPeaks(sigs, sig_cube, sig_sums, sig_folders, overwrite=True)
    gp.saveFile(f"Final_GPs/MC_{_id}")

  0%|          | 0/101 [00:00<?, ?it/s]